In [2]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [5]:
# cmap = plt.get_cmap('inferno')

# plt.figure(figsize=(10,10))
# genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
# for g in genres:
#     pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
#     for filename in os.listdir(f'./MIR/genres/{g}'):
#         songname = f'./MIR/genres/{g}/{filename}'
#         y, sr = librosa.load(songname, mono=True, duration=5)
#         plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
#         plt.axis('off');
#         plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
#         plt.clf()

In [6]:
# somehow we convert the songs to data format and we get data.csv
# don't ask its cery complicated

In [7]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00081.au,0.380260,0.248262,2116.942959,1956.611056,4196.107960,0.127272,-26.929785,107.334008,-46.809993,...,14.336612,-13.821769,7.562789,-6.181372,0.330165,-6.829571,0.965922,-7.570825,2.918987,blues
1,blues.00022.au,0.306451,0.113475,1156.070496,1497.668176,2170.053545,0.058613,-233.860772,136.170239,3.289490,...,-2.250578,3.959198,5.322555,0.812028,-1.107202,-4.556555,-2.436490,3.316913,-0.608485,blues
2,blues.00031.au,0.253487,0.151571,1331.073970,1973.643437,2900.174130,0.042967,-221.802549,110.843071,18.620984,...,-13.037723,-12.652228,-1.821905,-7.260097,-6.660252,-14.682694,-11.719264,-11.025216,-13.387260,blues
3,blues.00012.au,0.269320,0.119072,1361.045467,1567.804596,2739.625101,0.069124,-207.208080,132.799175,-15.438986,...,-0.613248,0.384877,2.605128,-5.188924,-9.527455,-9.244394,-2.848274,-1.418707,-5.932607,blues
4,blues.00056.au,0.391059,0.137728,1811.076084,2052.332563,3927.809582,0.075480,-145.434568,102.829023,-12.517677,...,7.457218,-10.470444,-2.360483,-6.783624,2.671134,-4.760879,-0.949005,0.024832,-2.005315,blues


In [8]:
data.shape

(1000, 28)

In [9]:
data = data.drop(['filename'], axis=1)

In [10]:
# encoding the labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [11]:
# scaling the feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:,:-1], dtype=float))

In [12]:
# Dividing data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
len(y_train)

800

In [14]:
len(y_test)

200

In [15]:
X_train[10]

array([ 1.20226572, -0.37578329,  0.56738133,  0.28452656,  0.61219829,
        0.56447527,  0.30154957, -0.59691913, -0.42657558,  0.86245738,
       -1.12773528,  1.2357794 , -1.50105885,  0.80891715, -1.2908672 ,
        1.12623363, -0.79088872,  1.76102346,  0.05356632,  1.75365697,
       -0.03979831,  1.74674674,  0.24680884,  0.03238475,  0.08913079,
        0.43433621])

In [16]:
# classification with keras
# building our network

from keras import models
from keras import layers

model= models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [17]:
model.compile(optimizer='adam', loss= 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
history = model.fit(X_train, y_train, epochs=20, batch_size=128)

W0328 13:22:41.923965  7892 deprecation_wrapper.py:119] From C:\installs\Anaconda\envs\tf_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/20
800/800 [==============================] - 1s 1ms/step - loss: 2.2099 - accuracy: 0.2450
Epoch 2/20
800/800 [==============================] - 0s 34us/step - loss: 1.9049 - accuracy: 0.3550
Epoch 3/20
800/800 [==============================] - 0s 43us/step - loss: 1.6957 - accuracy: 0.4013
Epoch 4/20
800/800 [==============================] - 0s 33us/step - loss: 1.5072 - accuracy: 0.4588
Epoch 5/20
800/800 [==============================] - 0s 29us/step - loss: 1.3572 - accuracy: 0.5400
Epoch 6/20
800/800 [==============================] - 0s 34us/step - loss: 1.2252 - accuracy: 0.6025
Epoch 7/20
800/800 [==============================] - 0s 32us/step - loss: 1.1242 - accuracy: 0.6350
Epoch 8/20
800/800 [==============================] - 0s 27us/step - loss: 1.0340 - accuracy: 0.6612
Epoch 9/20
800/800 [==============================] - 0s 35us/step - loss: 0.9771 - accuracy: 0.6825
Epoch 10/20
800/800 [==============================] - 0s 31us/step - loss: 0.9095 - accurac

In [20]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('test-acc: ', test_acc)

200/200 [==============================] - 0s 156us/step
test-acc:  0.6949999928474426


In [21]:
# test accuracy is low because of ack of validation set so we would set apart some data for validation

In [22]:
# validating by setting apart 200 samples from training data
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [24]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(partial_x_train, partial_y_train, epochs=30, batch_size=512,validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 0s 246us/step - loss: 2.2797 - accuracy: 0.1233 - val_loss: 2.1482 - val_accuracy: 0.3150
Epoch 2/30
600/600 [==============================] - 0s 18us/step - loss: 2.1001 - accuracy: 0.3317 - val_loss: 2.0362 - val_accuracy: 0.3100
Epoch 3/30
600/600 [==============================] - 0s 19us/step - loss: 1.9625 - accuracy: 0.3383 - val_loss: 1.9501 - val_accuracy: 0.3400
Epoch 4/30
600/600 [==============================] - 0s 21us/step - loss: 1.8583 - accuracy: 0.3367 - val_loss: 1.8789 - val_accuracy: 0.3200
Epoch 5/30
600/600 [==============================] - 0s 20us/step - loss: 1.7678 - accuracy: 0.3350 - val_loss: 1.7810 - val_accuracy: 0.3600
Epoch 6/30
600/600 [==============================] - 0s 22us/step - loss: 1.6654 - accuracy: 0.3833 - val_loss: 1.6833 - val_accuracy: 0.4550
Epoch 7/30
600/600 [==============================] - 0s 18us/step - loss: 1.5677 - accuracy: 0

In [25]:
results

[0.9906969928741455, 0.675000011920929]

In [26]:
predictions = model.predict(X_test)

In [28]:
predictions[0].shape

(10,)

In [30]:
print(predictions)
print(np.sum(predictions[0]))
print(np.argmax(predictions[0]))

[[8.3869651e-02 1.0167875e-05 8.9212883e-01 ... 1.2377385e-04
  1.1388448e-03 4.9406947e-03]
 [2.1782271e-03 3.4183080e-07 4.4865028e-05 ... 1.8030749e-05
  3.2828026e-04 1.7123594e-03]
 [3.1700727e-02 6.5950495e-03 2.7629334e-02 ... 1.5663324e-03
  3.0447964e-02 6.0170457e-02]
 ...
 [9.2790294e-03 3.1246752e-03 5.2448469e-03 ... 1.7961649e-03
  9.5393473e-01 3.5338763e-03]
 [6.1433192e-04 1.3812691e-05 9.2114201e-03 ... 3.4635805e-02
  8.3857387e-02 2.2899549e-02]
 [3.6458281e-01 1.8361963e-03 3.1628039e-01 ... 2.1134419e-02
  1.3304234e-01 7.3935397e-02]]
1.0
2
